In [1]:
import Base: iterate,max, exp, sin, cos, tan, +, ^, -, *, /, sqrt, convert, promote_rule, zero,isless
using BenchmarkTools
import Test: @test, @testset

In [2]:
2^2

4

# Define dual number

In [3]:
struct Dual{T <: Number} <: Number
    x::T
    dx::T
end
Dual(n::Integer, d::Float64) = Dual(promote(n, d)...)
Dual(n::Float64, d::Integer) = Dual(promote(n, d)...)

Dual

In [4]:
function convert(::Type{Dual{T}}, x::T) where {T}
   Dual(x, zero(x)) 
end
function convert(::Type{Dual{T}}, x::Dual{S}) where {S, T}
    Dual(T(x.x), T(x.dx))
end
# This is needed according to an error before
function convert(::Type{Dual{T}}, x::T) where {T <: Number}
    Dual(x, zero(x))
end
function convert(::Type{Dual{T}}, x::S) where {T, S <: Number}
    x_as_T = convert(T, x)
    Dual(x_as_T, zero(x_as_T))
end

convert (generic function with 191 methods)

In [5]:
function zero(x::Dual{T}) where T
    Dual(zero(x.x), zero(x.dx))
end

zero (generic function with 23 methods)

In [6]:
function promote_rule(::Type{Dual{T}}, ::Type{Dual{S}}) where {T,S}
    Dual{promote_type(T,S)}
end
function promote_rule(::Type{Dual{T}}, ::Type{S}) where {T, S <: Number}
    Dual{promote_type(T,S)}
end
function promote_rule(::Type{T}, ::Type{Dual{S}}) where {T <: Number, S}
    Dual{promote_type(T,S)}
end
function promote_rule(::Type{S}, ::Type{Dual{T}}) where {S <: AbstractIrrational, T}
    Dual{promote_type(S,T)}
end

promote_rule (generic function with 128 methods)

In [7]:
function extract_derivative(xdx::Dual)
    return xdx.dx
end
function extract_derivative(xs::Array)
    [extract_derivative(x) for x in xs]
end
function extract_derivative(xs::Tuple)
    convert(Tuple, [extract_derivative(x) for x in xs])
end

extract_derivative (generic function with 3 methods)

# Differential operator

In [8]:
function derivativeСalculation(f, value)
    function df(x)
        xdx = Dual(x, one(x))
        result = f(xdx)
        return extract_derivative(result)
    end
    df(value)
end
function derivativeСalculation(f,i::Integer,values)
    function df(valuesS)
        xarr = [(j != i ? x : Dual(x, one(x))) for (j,x) in enumerate(valuesS)]
        result = f(xarr...)
        return extract_derivative(result)
    end
    df(values)
end
function derivativeСalculation(f,values)
    function df(xs...)
        xarr = [(j != i ? x : Dual(x, one(x))) for (j,x) in enumerate(xs)]
        result = f(xarr...)
        return extract_derivative(result)
    end
    df(values)
end

derivativeСalculation (generic function with 2 methods)

In [9]:
function +(x::Dual, y::Dual)
    Dual(x.x+y.x, x.dx+y.dx)
end
function -(x::Dual, y::Dual)
    Dual(x.x-y.x, x.dx-y.dx)
end
function -(x::Dual)
    Dual(-x.x, -x.dx)
end
function *(x::Dual, y::Dual)
#      println("mnożenie")
     Dual(x.x*y.x, x.x*y.dx + x.dx*y.x)
end
function /(x::Dual, y::Dual)
    Dual(x.x/y.x, x.dx/y.x - x.x*y.dx/(y.x*y.x))
end
function /(x::Array, y::Number)
    return [i / y for i in x]
end

function ^(a::Dual, x::Dual)
#     println("Daszek")
    Dual(a.x^x.x, a.dx * x.x * a.x ^ (x.x - 1) + x.dx * a.x ^ x.x * log(a.x)) 
end

function ^(a::Dual, x::Integer)
#     println("Daszek")
    Dual(a.x^x, a.dx * x * a.x ^ (x - 1)) 
end

^ (generic function with 68 methods)

In [10]:
function sqrt(x::Dual)
    Dual(sqrt(x.x), x.dx/(2*sqrt(x.x)))
end

sqrt (generic function with 20 methods)

In [11]:
function exp(x::Dual)
    return Dual(exp(x.x), exp(x.x)*x.dx)
end
function exp(xs::Array)
    print(xs)
    return [exp(x) for x in xs]
end
function sin(x::Dual)
    return Dual(sin(x.x), cos(x.x)*x.dx)
end
function cos(x::Dual)
    return Dual(cos(x.x), -sin(x.x)*x.dx)
end

function tan(x::Dual)
    return Dual(tan(x.x),(1/cos(x.x)^2)*x.dx)
end
isless(x::Dual, y::Dual) = x.x < y.x;

In [12]:
function max(x::Dual)
    return Dual(max(0,x.x), x.x < 0 ? 0 : 1 * x.dx)
end

max (generic function with 14 methods)

In [13]:
function softmax(vector::Array)
    e = exp(vector)
    return e / sum(e)
#     z = [Dual(2.0, 1.0), Dual(21.0, 1.0)]
#     s = zero(vector[1])
#     for x in z
#        s = s + x 
#     end    
#     return e / s
end

softmax (generic function with 1 method)

In [14]:
function iterate(iter::Vector{Dual}, state=1)
#     println(iter, "Test22232323")
    if state > length(iter)
        return nothing
    end
    return (iter[state],state+1)
end

iterate (generic function with 228 methods)

In [15]:
J = function jacobian(f, args::Vector{T}) where {T <:Number}
    jacobian_columns = Matrix{T}[]
    
    for i=1:length(args)
        x = Dual{T}[]
        for j=1:length(args)
            seed = (i == j)
            push!(x, seed ?
                Dual(args[j], one(args[j])) :
                Dual(args[j],zero(args[j])) )
        end
        temp  = [f(x)...]
        column = extract_derivative.([f(x)...])
        push!(jacobian_columns, column[:,:])
    end
    hcat(jacobian_columns...)
end

jacobian (generic function with 1 method)

In [16]:
f(x::Vector) = softmax(x)
J(softmax, [1., 2.])

Dual{Float64}[Dual{Float64}(1.0, 1.0), Dual{Float64}(2.0, 0.0)]Dual{Float64}[Dual{Float64}(1.0, 1.0), Dual{Float64}(2.0, 0.0)]Dual{Float64}[Dual{Float64}(1.0, 0.0), Dual{Float64}(2.0, 1.0)]Dual{Float64}[Dual{Float64}(1.0, 0.0), Dual{Float64}(2.0, 1.0)]

2×2 Matrix{Float64}:
  0.196612  -0.196612
 -0.196612   0.196612

In [17]:
f(x::Vector) = [x[1], 5/x[3], 4x[2]^2-2x[3], x[3]*sin(x[1]), exp(x[1])/sum(x)]
J(f, [1, 2., 3])

5×3 Matrix{Float64}:
 1.0        0.0         0.0
 0.0        0.0        -0.555556
 0.0       16.0        -2.0
 1.62091    0.0         0.841471
 0.377539  -0.0755078  -0.0755078

In [18]:
function linear(x)
    return x^2
end


linear (generic function with 1 method)

In [19]:
derivativeСalculation(linear,1,[12])

24

In [20]:
softmax([1,2,3])

[1, 2, 3]

3-element Vector{Float64}:
 0.09003057317038046
 0.24472847105479767
 0.6652409557748219

In [21]:
function relu(x)
    return max(0,x)
end

relu (generic function with 1 method)

In [22]:
function rosenbrock(x::Vector)
    value = zero(x[1])
    for i=2:length(x)
        value += (1-x[i-1])^2 + 100*(x[i] - x[i-1]^2)^2
    end
    value
end


rosenbrock (generic function with 1 method)

In [23]:
# d = derivativeСalculation(rosenbrock,[1.0,2.0,3.0])
@benchmark J(rosenbrock, [rand(1000,1)...])

BenchmarkTools.Trial: 
  memory estimate:  32.11 MiB
  allocs estimate:  15018
  --------------
  minimum time:     43.212 ms (0.00% GC)
  median time:      44.108 ms (1.45% GC)
  mean time:        44.209 ms (1.13% GC)
  maximum time:     47.521 ms (0.00% GC)
  --------------
  samples:          114
  evals/sample:     1

In [24]:
[rand(3,1)...]

3-element Vector{Float64}:
 0.45729765595003324
 0.9551764265774303
 0.5506191695086682

In [25]:
s = rand(3,1)
print(s[1][1])

0.6475895176869497

In [26]:
J(rosenbrock, [1., 2., 3, 4, 5,6 ,7])

1×7 Matrix{Float64}:
 -400.0  1002.0  5804.0  16606.0  35808.0  65810.0  -5800.0

In [27]:
rosenbrock([1,2])

100

In [28]:
function rosenbrock(x)
    value = zero(x[1])
#     value = (1-a)^2 + 100*(x - a^2)^2
    for i=2:length(x)
        value += (1-x[i-1])^2 + 100*(x[i] - x[i-1]^2)^3
    end
    value
end

rosenbrock (generic function with 2 methods)

In [29]:
derivativeСalculation(rosenbrock,1,[1,2,3])

LoadError: MethodError: no method matching rosenbrock(::Dual{Int64}, ::Int64, ::Int64)
[0mClosest candidates are:
[0m  rosenbrock(::Any) at In[28]:1

In [30]:
tan(2)

-2.185039863261519

In [31]:
f(x::Vector) = [x[1], 5/x[3], 4x[2]^2-2x[3], x[3]*sin(x[1]), exp(x[1])/sum(x), x[1]^2 - x[2]^3 - 88]
z = @benchmark J(f,[rand(3,1)...]) 

BenchmarkTools.Trial: 
  memory estimate:  5.27 KiB
  allocs estimate:  79
  --------------
  minimum time:     3.933 μs (0.00% GC)
  median time:      4.041 μs (0.00% GC)
  mean time:        4.394 μs (5.21% GC)
  maximum time:     289.774 μs (97.92% GC)
  --------------
  samples:          10000
  evals/sample:     8

In [32]:
J(f, [2.9,2.2,1546.23])

6×3 Matrix{Float64}:
     1.0          0.0          0.0
     0.0          0.0         -2.09133e-6
     0.0         17.6         -2.0
 -1501.32         0.0          0.239249
     0.0117077   -7.55172e-6  -7.55172e-6
     5.8        -14.52         0.0

In [33]:
println(z.params)


BenchmarkTools.Parameters(5.0, 10000, 8, 0.0, true, false, 0.05, 0.01)


In [34]:
dump(z)
rand(3)

BenchmarkTools.Trial
  params: BenchmarkTools.Parameters
    seconds: Float64 5.0
    samples: Int64 10000
    evals: Int64 8
    overhead: Float64 0.0
    gctrial: Bool true
    gcsample: Bool false
    time_tolerance: Float64 0.05
    memory_tolerance: Float64 0.01
  times: Array{Float64}((10000,)) [3933.375, 3935.0, 3937.875, 3942.875, 3943.125, 3944.0, 3944.5, 3944.875, 3946.625, 3946.625  …  11986.75, 247868.625, 249471.25, 250823.5, 253074.125, 253084.125, 256742.25, 260741.875, 271286.25, 289774.0]
  gctimes: Array{Float64}((10000,)) [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 243103.5, 244699.125, 245910.625, 248343.75, 248148.125, 251971.75, 255965.125, 266294.125, 283754.0]
  memory: Int64 5392
  allocs: Int64 79


3-element Vector{Float64}:
 0.05492853965120337
 0.09530128421030515
 0.15436688597484238

## Definicja funkcji testowych

In [35]:
function styblinskiTang(x)
    value = zero(x[1])
    for i=1:length(x)
        value += x[i]^4 - 16 * x[i]^2 + 5 * x[i]
    end
    value / 2
end

function rosenbrock(x)
    value = zero(x[1])
    for i=2:length(x)
        value += (1-x[i-1])^2 + 100*(x[i] - x[i-1]^2)^2
    end
    value
end

function KarolikJeleniewiczFunction(x)
    f1 = 222.83*(x[2]^8+x[3]^-3)^2 + sin(x[5]^18) - cos(x[1])/exp(x[3])
    f2 = x[1] - 5/x[2] + 4*x[3]^2-2*x[4] / (x[3]*sin(x[5])^2 + 1)
    f3 = sqrt(exp(x[2])^2) + x[3]^x[2] - x[4]
    [f1,f2,f3]
end

function viennet(x)
    f1 = 0.5*(x[1]^2+x[2]^2) + sin(x[1]^2+ x[2]^2)
    f2 = (x[1]-2*x[2]+4)^2/8 + (x[1]-x[2]+1)^2/27 + 15
    f3 = 1/(x[1]^2+x[2]^2+1) - 1.1* exp(-(x[1]^2+x[2]^2))
    [f1,f2,f3]
end


viennet (generic function with 1 method)

## Testy funkcji weluzmiennych 

### Testowanie funkcji Rosenbrock

In [36]:
J(rosenbrock, [5.3,2.2,34.2,123.1])

1×4 Matrix{Float64}:
 54895.4  -31012.4  1.43226e7  -209308.0

In [37]:
@benchmark J(rosenbrock, [rand(5,1)...])

BenchmarkTools.Trial: 
  memory estimate:  4.11 KiB
  allocs estimate:  48
  --------------
  minimum time:     2.834 μs (0.00% GC)
  median time:      2.984 μs (0.00% GC)
  mean time:        3.316 μs (7.99% GC)
  maximum time:     378.986 μs (98.84% GC)
  --------------
  samples:          10000
  evals/sample:     9

In [38]:
@benchmark J(rosenbrock, [rand(50,1)...])

BenchmarkTools.Trial: 
  memory estimate:  130.81 KiB
  allocs estimate:  561
  --------------
  minimum time:     119.226 μs (0.00% GC)
  median time:      120.441 μs (0.00% GC)
  mean time:        129.016 μs (3.42% GC)
  maximum time:     2.038 ms (93.35% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [39]:
@benchmark J(rosenbrock, [rand(100,1)...])

BenchmarkTools.Trial: 
  memory estimate:  467.36 KiB
  allocs estimate:  1212
  --------------
  minimum time:     461.219 μs (0.00% GC)
  median time:      469.946 μs (0.00% GC)
  mean time:        493.868 μs (3.13% GC)
  maximum time:     3.298 ms (80.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [40]:
@benchmark J(rosenbrock, [rand(500,1)...])

BenchmarkTools.Trial: 
  memory estimate:  8.20 MiB
  allocs estimate:  7014
  --------------
  minimum time:     10.926 ms (0.00% GC)
  median time:      11.055 ms (0.00% GC)
  mean time:        11.237 ms (1.62% GC)
  maximum time:     12.861 ms (8.16% GC)
  --------------
  samples:          445
  evals/sample:     1

In [41]:
@benchmark J(rosenbrock, [rand(1000,1)...])

BenchmarkTools.Trial: 
  memory estimate:  32.11 MiB
  allocs estimate:  15018
  --------------
  minimum time:     43.243 ms (0.00% GC)
  median time:      44.255 ms (1.64% GC)
  mean time:        44.794 ms (1.34% GC)
  maximum time:     48.740 ms (2.20% GC)
  --------------
  samples:          112
  evals/sample:     1

In [42]:
@benchmark J(rosenbrock, [rand(5000,1)...])

BenchmarkTools.Trial: 
  memory estimate:  1.23 GiB
  allocs estimate:  90024
  --------------
  minimum time:     1.098 s (1.26% GC)
  median time:      1.107 s (1.42% GC)
  mean time:        1.115 s (1.44% GC)
  maximum time:     1.141 s (1.59% GC)
  --------------
  samples:          5
  evals/sample:     1

In [43]:
@benchmark J(rosenbrock, [rand(10000,1)...])

BenchmarkTools.Trial: 
  memory estimate:  4.89 GiB
  allocs estimate:  190025
  --------------
  minimum time:     4.785 s (1.64% GC)
  median time:      4.808 s (1.66% GC)
  mean time:        4.808 s (1.66% GC)
  maximum time:     4.831 s (1.67% GC)
  --------------
  samples:          2
  evals/sample:     1

### Testowanie funkcji styblinskiTang

In [44]:
J(styblinskiTang, [5.3,2.2,34.2,123.1])

1×4 Matrix{Float64}:
 215.454  -11.404  79458.7  3.72885e6

In [45]:
@benchmark J(styblinskiTang, [rand(5,1)...])

BenchmarkTools.Trial: 
  memory estimate:  4.11 KiB
  allocs estimate:  48
  --------------
  minimum time:     2.905 μs (0.00% GC)
  median time:      2.993 μs (0.00% GC)
  mean time:        3.472 μs (8.94% GC)
  maximum time:     499.458 μs (98.90% GC)
  --------------
  samples:          10000
  evals/sample:     9

In [46]:
@benchmark J(styblinskiTang, [rand(50,1)...])

BenchmarkTools.Trial: 
  memory estimate:  130.81 KiB
  allocs estimate:  561
  --------------
  minimum time:     85.789 μs (0.00% GC)
  median time:      90.906 μs (0.00% GC)
  mean time:        96.727 μs (4.83% GC)
  maximum time:     2.191 ms (95.48% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [47]:
@benchmark J(styblinskiTang, [rand(100,1)...])

BenchmarkTools.Trial: 
  memory estimate:  467.36 KiB
  allocs estimate:  1212
  --------------
  minimum time:     332.996 μs (0.00% GC)
  median time:      339.232 μs (0.00% GC)
  mean time:        358.546 μs (4.26% GC)
  maximum time:     2.302 ms (79.92% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [48]:
@benchmark J(styblinskiTang, [rand(500,1)...])

BenchmarkTools.Trial: 
  memory estimate:  8.20 MiB
  allocs estimate:  7014
  --------------
  minimum time:     7.328 ms (0.00% GC)
  median time:      7.626 ms (0.00% GC)
  mean time:        7.804 ms (2.45% GC)
  maximum time:     10.570 ms (12.90% GC)
  --------------
  samples:          641
  evals/sample:     1

In [49]:
@benchmark J(styblinskiTang, [rand(1000,1)...])

BenchmarkTools.Trial: 
  memory estimate:  32.11 MiB
  allocs estimate:  15018
  --------------
  minimum time:     28.744 ms (0.00% GC)
  median time:      29.704 ms (2.16% GC)
  mean time:        29.963 ms (1.74% GC)
  maximum time:     33.625 ms (3.84% GC)
  --------------
  samples:          167
  evals/sample:     1

In [50]:
@benchmark J(styblinskiTang, [rand(5000,1)...])

BenchmarkTools.Trial: 
  memory estimate:  1.23 GiB
  allocs estimate:  90024
  --------------
  minimum time:     737.323 ms (1.85% GC)
  median time:      764.241 ms (2.10% GC)
  mean time:        760.267 ms (2.09% GC)
  maximum time:     787.906 ms (2.37% GC)
  --------------
  samples:          7
  evals/sample:     1

In [51]:
@benchmark J(styblinskiTang, [rand(10000,1)...])

BenchmarkTools.Trial: 
  memory estimate:  4.89 GiB
  allocs estimate:  190025
  --------------
  minimum time:     3.107 s (2.39% GC)
  median time:      3.124 s (2.36% GC)
  mean time:        3.124 s (2.36% GC)
  maximum time:     3.142 s (2.32% GC)
  --------------
  samples:          2
  evals/sample:     1

## Testy funkcji wektorowych

In [42]:
@benchmark J(viennet, [rand(2,1)...])

BenchmarkTools.Trial: 
  memory estimate:  2.72 KiB
  allocs estimate:  41
  --------------
  minimum time:     2.290 μs (0.00% GC)
  median time:      2.385 μs (0.00% GC)
  mean time:        2.602 μs (5.19% GC)
  maximum time:     294.911 μs (98.73% GC)
  --------------
  samples:          10000
  evals/sample:     9

In [61]:
@benchmark J(KarolikJeleniewiczFunction, [rand(5,1)...])

BenchmarkTools.Trial: 
  memory estimate:  7.16 KiB
  allocs estimate:  103
  --------------
  minimum time:     10.838 μs (0.00% GC)
  median time:      11.674 μs (0.00% GC)
  mean time:        12.357 μs (2.18% GC)
  maximum time:     2.709 ms (99.32% GC)
  --------------
  samples:          10000
  evals/sample:     1

# Testowanie

In [37]:
J(KarolikJeleniewiczFunction, [2.9,2.2,2.2,34456.5,1.8])

3×5 Matrix{Float64}:
 0.0265096   4.88099e8  -31325.0      0.0           1.77884e5
 1.0         1.03306      6878.31    -0.647997  -7042.76
 0.0        13.493           5.6667  -1.0           0.0

In [38]:
D = Dict("viennet" => [viennet, 2], "KarolikJeleniewiczFunction" => [KarolikJeleniewiczFunction, 5])

Dict{String, Vector{Any}} with 2 entries:
  "viennet"                    => [viennet, 2]
  "KarolikJeleniewiczFunction" => [KarolikJeleniewiczFunction, 5]

In [39]:
    for key in keys(D)
           informationAboutFunction = get(D, key, Nothing)
            println(informationAboutFunction[2])
    end

2
5


In [40]:
function makeTestWektorFunction(definition)
    benchamarkResults = []
    for key in keys(definition)
        print(key)
       informationAboutFunction = get(definition, key, Nothing)
        println(informationAboutFunction[2])
       push!(benchamarkResults, d(viennet,[rand(is[2],1)...])))
#         print(J(informationAboutFunction[1],[rand(informationAboutFunction[2],1)...]))
    end
    benchamarkResults
end


LoadError: syntax: "for" at In[40]:3 expected "end", got ")"

In [41]:
x = makeTestWektorFunction(D)


LoadError: UndefVarError: makeTestWektorFunction not defined